### Gathering and Cleaning Data to get one DataFrame
This file gathers data from fred and other .csv imported files and then cleans and merges them all together to get one final dataframe we can use for the rest of the report. The result of this file is a **commodities_df.csv** file that is exported to the **input_data** folder.

#### Imports

In [1]:
import pandas as pd
import numpy as np
import pandas_datareader as pdr 
import datetime
import re

#import warnings
#warnings.filterwarnings("ignore")

#### Load Data and Create a Final Corn DataFrame

In [2]:
#Load .csv files that contain necessary data 
cornFutures = pd.read_csv('input_data/futures/us_corn_monthly.csv')
cornClimate = pd.read_csv('input_data/climate/cornClimate.csv')
sp500 = pd.read_csv('input_data/macroeconomic/sp500Monthly.csv')

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (19,23,27,39,41,45) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
#Convert Date columns to datetime so that they can be merged
cornFutures['Date_Formatted'] = cornFutures['Date'][0:268].str[:3] + '-20' + cornFutures['Date'][0:268].str[4:]
cornFutures['Date_Formatted'][268:] = cornFutures['Date'][268:].str[:3] + '-19' + cornFutures['Date'][268:].str[4:]
cornFutures['Date_Formatted'] = pd.to_datetime(cornFutures['Date_Formatted'])
cornFutures = cornFutures[['Date_Formatted', 'Price']]

/var/folders/g5/71w7b_bn4vd9prdmhdgdzryr0000gn/T/ipykernel_27758/380913680.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cornFutures['Date_Formatted'][268:] = cornFutures['Date'][268:].str[:3] + '-19' + cornFutures['Date'][268:].str[4:]


In [4]:
#Date format flips between row # 228 & 229
sp500['DATE'] = sp500['Date'][:121].str[-3:] + '-20' + sp500['Date'][:121].str[:2]
sp500['DATE'][121:229] = sp500['Date'][121:229].str[-3:] + '-200' + sp500['Date'][121:229].str[:1]
sp500['DATE'][229:241] = sp500['Date'][229:241].str[:3] + '-20' + sp500['Date'][229:241].str[-2:]
sp500['DATE'][241:] = sp500['Date'][241:].str[:3] + '-19' + sp500['Date'][241:].str[-2:]

sp500['DATE'] = pd.to_datetime(sp500['DATE'])

sp500 = sp500[['DATE', 'Price']]
sp500.rename(columns = {'Price':'sp500_Price'}, inplace = True)
#sp500 = sp500.sort_values('Date')
sp500 = sp500.iloc[::-1]

In [5]:
#Begin by scraping macroeconomic data from fred, including GDP, CPI, and UNRATE
start = datetime.datetime(1990, 1, 1) 
end = datetime.datetime(2022, 2, 28)

macro_df = pdr.data.DataReader(['GDP','CPIAUCSL','UNRATE'], 'fred', start, end)
macro_df = macro_df.reset_index()
macro_df = macro_df.loc[macro_df['DATE'].dt.day == 1] #takes first day out of each month

In [6]:
#Merge the corn futures data and corn climate data 
macro_corn_df = macro_df.merge(cornFutures, 
                               how='left', 
                               left_on='DATE', 
                               right_on='Date_Formatted') #merges corn data with macro data

cornClimate = cornClimate[['DATE', 'PRCP', 'SNOW', 'TMAX', 'TMIN']]
cornClimate['DATE'] = pd.to_datetime(cornClimate['DATE'])

Corn_Final = macro_corn_df.merge(cornClimate, how='left', on = 'DATE')
Corn_Final.rename(columns = {'Price':'Corn_Future_Price'}, inplace = True)

In [7]:
#Merge the S&P500 data, which is the last set of data we need for the final DF

Corn_Final = Corn_Final.merge(sp500, on = 'DATE', how='right')

Corn_Final['GDP'] = Corn_Final['GDP'].interpolate(method='nearest') #fill in Missing GDPs with nearest value

Corn_Final = Corn_Final.drop('Date_Formatted', 1)

/var/folders/g5/71w7b_bn4vd9prdmhdgdzryr0000gn/T/ipykernel_27758/160183632.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Corn_Final = Corn_Final.drop('Date_Formatted', 1)


#### Create Final Wheat DataFrame

In [8]:
#Load wheat csv files
wheatFutures = pd.read_csv('input_data/futures/us_wheat_monthly.csv')
wheatClimate = pd.read_csv('input_data/climate/wheatClimate.csv')

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (19,21,25,27,29,33,49,51,57,59) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
#Convert Date columns to datetime so that they can be merged

wheatFutures['DATE'] = wheatFutures['Date'][0:268].str[:3] + '-20' + wheatFutures['Date'][0:268].str[4:]
wheatFutures['DATE'][268:] = wheatFutures['Date'][268:].str[:3] + '-19' + wheatFutures['Date'][268:].str[4:]
wheatFutures['DATE'] = pd.to_datetime(wheatFutures['DATE'])
wheatFutures = wheatFutures[['DATE', 'Price']]

/var/folders/g5/71w7b_bn4vd9prdmhdgdzryr0000gn/T/ipykernel_27758/940890169.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wheatFutures['DATE'][268:] = wheatFutures['Date'][268:].str[:3] + '-19' + wheatFutures['Date'][268:].str[4:]


In [10]:
#Get wheat climate data ready
wheatClimate = wheatClimate[['DATE', 'PRCP', 'SNOW', 'TMAX', 'TMIN']]
wheatClimate['DATE'] = pd.to_datetime(wheatClimate['DATE'])

In [11]:
#Merge the wheat futures data and wheat climate data 
macro_wheat_df = macro_df.merge(wheatFutures, 
                               how='left', 
                               on='DATE') #merges wheat data with macro data


Wheat_Final = macro_wheat_df.merge(wheatClimate, how='left', on = 'DATE')
Wheat_Final.rename(columns = {'Price':'Wheat_Future_Price'}, inplace = True)

In [12]:
#Merge in the S&P500 data

Wheat_Final = Wheat_Final.merge(sp500, on = 'DATE', how='right')

Wheat_Final['GDP'] = Wheat_Final['GDP'].interpolate(method='nearest') #fill in Missing GDPs with nearest value

#### Create Final Soybeans DataFrame

In [13]:
#Load wheat csv files
soybeansFutures = pd.read_csv('input_data/futures/us_soybeans_monthly.csv')
soybeansClimate = pd.read_csv('input_data/climate/soybeansClimate.csv')

In [14]:
#Convert Date columns to datetime so that they can be merged

soybeansFutures['DATE'] = soybeansFutures['Date'][0:268].str[:3] + '-20' + soybeansFutures['Date'][0:268].str[4:]
soybeansFutures['DATE'][268:] = soybeansFutures['Date'][268:].str[:3] + '-19' + soybeansFutures['Date'][268:].str[4:]
soybeansFutures['DATE'] = pd.to_datetime(soybeansFutures['DATE'])
soybeansFutures = soybeansFutures[['DATE', 'Price']]

In [15]:
#Get soybeans climate data ready
soybeansClimate = soybeansClimate[['DATE', 'PRCP', 'SNOW', 'TMAX', 'TMIN']]
soybeansClimate['DATE'] = pd.to_datetime(soybeansClimate['DATE'])

In [16]:
#Merge the soybeans futures data and soybeans climate data 
macro_soybeans_df = macro_df.merge(soybeansFutures, 
                               how='left', 
                               on='DATE') #merges wheat data with macro data


Soybeans_Final = macro_soybeans_df.merge(soybeansClimate, how='left', on = 'DATE')
Soybeans_Final.rename(columns = {'Price':'Soybeans_Future_Price'}, inplace = True)

In [17]:
#Merge in the S&P500 data

Soybeans_Final = Soybeans_Final.merge(sp500, on = 'DATE', how='right')

Soybeans_Final['GDP'] = Soybeans_Final['GDP'].interpolate(method='nearest') #fill in Missing GDPs with nearest value

#### Create One Final Commodity DataFrame

In [18]:
Corn_Final = Corn_Final.rename({'PRCP': 'C_PRCP', 
                                'SNOW': 'C_SNOW', 
                                'TMAX': 'C_TMAX',
                                'TMIN': 'C_TMIN'}, axis=1)


Soybeans_Final = Soybeans_Final.rename({'PRCP': 'S_PRCP', 
                                        'SNOW': 'S_SNOW', 
                                        'TMAX': 'S_TMAX',
                                        'TMIN': 'S_TMIN'}, axis=1)

Soybeans_Final = Soybeans_Final[['DATE','Soybeans_Future_Price',
                                 'S_PRCP','S_SNOW','S_TMAX','S_TMIN']]


Wheat_Final = Wheat_Final.rename({'PRCP': 'W_PRCP', 
                                  'SNOW': 'W_SNOW', 
                                  'TMAX': 'W_TMAX',
                                  'TMIN': 'W_TMIN'}, axis=1)

Wheat_Final = Wheat_Final[['DATE','Wheat_Future_Price','W_PRCP',
                           'W_SNOW','W_TMAX','W_TMIN']]


Commodities_Final = Corn_Final.merge(Soybeans_Final, on = 'DATE')

Commodities_Final = Commodities_Final.merge(Wheat_Final, on = 'DATE')

#Convert several price columns into float64 dtypes (were object dtypes)
Commodities_Final = Commodities_Final.replace(',','', regex=True) #replace commas b/c it interferes when changing to numeric types 

Commodities_Final['Soybeans_Future_Price'] = pd.to_numeric(Commodities_Final['Soybeans_Future_Price'], errors = 'coerce')
Commodities_Final['Wheat_Future_Price'] = pd.to_numeric(Commodities_Final['Wheat_Future_Price'], errors = 'coerce')
Commodities_Final['sp500_Price'] = pd.to_numeric(Commodities_Final['sp500_Price'], errors = 'coerce')


#Compute realized commodity returns
Commodities_Final['realized_ret_corn'] = (np.log(Commodities_Final['Corn_Future_Price'].shift(-1)) 
                                       - np.log(Commodities_Final['Corn_Future_Price']))

Commodities_Final['realized_ret_soybeans'] = (np.log(Commodities_Final['Soybeans_Future_Price'].shift(-1)) 
                                           - np.log(Commodities_Final['Soybeans_Future_Price']))

Commodities_Final['realized_ret_wheat'] = (np.log(Commodities_Final['Wheat_Future_Price'].shift(-1)) 
                                        - np.log(Commodities_Final['Wheat_Future_Price']))


Commodities_Final['ret_gdp'] = (np.log(Commodities_Final['GDP'].shift(-1)) 
                                        - np.log(Commodities_Final['GDP']))

Commodities_Final['ret_cpi'] = (np.log(Commodities_Final['CPIAUCSL'].shift(-1)) 
                                        - np.log(Commodities_Final['CPIAUCSL']))

#Finally, export the final .csv file 
Commodities_Final.to_csv('output_data/commodities_df.csv', index=False)

Commodities_Final

,DATE,GDP,CPIAUCSL,UNRATE,Corn_Future_Price,C_PRCP,C_SNOW,C_TMAX,C_TMIN,sp500_Price,...,Wheat_Future_Price,W_PRCP,W_SNOW,W_TMAX,W_TMIN,realized_ret_corn,realized_ret_soybeans,realized_ret_wheat,ret_gdp,ret_cpi
0,1990-01-01,5872.701,127.500,5.4,NaN,0.0,0.0,22.0,-33.0,329.08,...,NaN,5.0,0.0,39.0,-28.0,NaN,NaN,NaN,0.000000,0.003914
1,1990-02-01,5872.701,128.000,5.3,NaN,10.0,0.0,67.0,6.0,331.89,...,NaN,30.0,33.0,11.0,-28.0,NaN,NaN,NaN,0.014761,0.004677
2,1990-03-01,5960.028,128.600,5.2,NaN,0.0,0.0,67.0,-72.0,339.94,...,NaN,0.0,0.0,111.0,-17.0,NaN,NaN,NaN,0.000000,0.002330
3,1990-04-01,5960.028,128.900,5.4,281.25,3.0,0.0,161.0,72.0,330.80,...,NaN,0.0,0.0,194.0,44.0,-0.011623,-0.040740,NaN,0.000000,0.001550
4,1990-05-01,5960.028,129.100,5.4,278.00,0.0,0.0,156.0,56.0,361.23,...,NaN,0.0,0.0,139.0,17.0,0.061048,0.026408,NaN,0.009200,0.006178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,2019-09-01,21694.458,256.596,3.5,388.00,135.0,0.0,233.0,128.0,2976.74,...,495.50,0.0,NaN,322.0,117.0,0.005141,0.029959,0.026389,0.000000,0.002759
357,2019-10-01,21694.458,257.305,3.6,390.00,0.0,0.0,306.0,189.0,3037.56,...,508.75,0.0,NaN,100.0,6.0,-0.049271,-0.060991,0.063531,0.000000,0.001875
358,2019-11-01,21694.458,257.788,3.6,371.25,51.0,66.0,-17.0,-78.0,3140.98,...,542.12,0.0,NaN,78.0,-94.0,0.043485,0.073523,0.031342,-0.009871,0.001841
359,2019-12-01,21481.367,258.263,3.6,387.75,38.0,NaN,78.0,6.0,3230.78,...,559.38,0.0,NaN,-11.0,-72.0,-0.016905,-0.078096,-0.010784,0.000000,0.001621
